In [9]:

import numpy as np
import cupy as cp
import h5py
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from IPython.display import Image, display

In [10]:
f = h5py.File('./data/trainlist.h5', 'r')
trainX = f['trainX']
trainY = f['trainY']
trainX=cp.array(trainX)
trainY=cp.array(trainY)
print(trainX.shape)
print(trainY.shape)

(97, 3, 20, 20, 7)
(97, 20, 20, 1)


In [11]:
#对每列进行归一化
def normalize(data):
    for i in range(data.shape[-1]):
        print(i,'列最大值为：',data[:,:,:,i].max())
        data[:,:,:,:,i] = data[:,:,:,:,i]/(int(data[:,:,:,i].max())+1)
    return data
trainX=normalize(trainX)
trainY=trainY/2977
    

0 列最大值为： 2376.245
1 列最大值为： 2976.428
2 列最大值为： 4734.875
3 列最大值为： 1063.0
4 列最大值为： 115.0
5 列最大值为： 151.0
6 列最大值为： 71.0


In [8]:
# Construct the input layer with no definite frame size.
inp = layers.Input(shape=(None,20,20,7))

# We will construct 3 `ConvLSTM2D` layers with batch normalization,
# followed by a `Conv3D` layer for the spatiotemporal outputs.
x = layers.ConvLSTM2D(
    filters=16,
    kernel_size=(3, 3),
    padding="same",
    return_sequences=True,
    activation="relu",
)(inp)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=16,
    kernel_size=(3, 3),
    padding="same",
    return_sequences=False,
    activation="relu",
)(x)
x = layers.Conv2D(
    filters=1, kernel_size=(1, 1), activation="sigmoid", padding="same"
)(x)

# Next, we will build the complete model and compile it.
model = keras.models.Model(inp, x)
model.summary()
model.compile(
    loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(),
)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None, 20, 20, 7)  0         
                             ]                                   
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, None, 20, 20, 16)  13312     
                                                                 
 batch_normalization_1 (Batc  (None, None, 20, 20, 16)  64       
 hNormalization)                                                 
                                                                 
 conv_lstm2d_3 (ConvLSTM2D)  (None, 20, 20, 16)        18496     
                                                                 
 conv2d_1 (Conv2D)           (None, 20, 20, 1)         17        
                                                                 
Total params: 31,889
Trainable params: 31,857
Non-trainable

In [9]:
from tensorflow.python import test


# Define modifiable training hyperparameters.
epochs = 200
batch_size = 2
log_dir = "/content/drive/Shareddrives/gsd/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# Fit the model to the training data.
model.fit(
    trainX.get(),
    trainY.get(),
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    callbacks=[tensorboard_callback],
    
)
model.save(os.path.join("/content/drive/Shareddrives/gsd/dataset_convlstm",f"Test_{kpi_name}" + ".h5"))

Epoch 1/200


: 

: 